# PART 1

## 1.Import packages and build a soup function

In [2]:
from bs4 import BeautifulSoup
import re
import requests
import bs4
import pandas as pd
origin = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(origin, 'lxml')

## 2.Get the table from the url

In [3]:
table = soup.find('table', class_ = "wikitable sortable")
df = pd.read_html(str(table))[0]
column_name = df.loc[[0]]
list = df.drop([0])
list.columns = column_name.iloc[0]
list.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


## 3. Ignore Borough column that is not assigned

In [4]:
# Ignore cells with a borough that is Not assigned
list1 = list[-list.Borough.isin(['Not assigned'])]
list1.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


## 4.Replace the Neighbourhood that is not assigned with Borough 

In [5]:
import numpy as np
list1['Neighbourhood'][list1['Neighbourhood'] == 'Not assigned'] = list1['Borough']
list1.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


## 5.Duplicate process

In [6]:
group = list1.groupby(['Postcode', 'Borough'], as_index = False).agg({'Neighbourhood': ['sum']})
group.head()

,Postcode,Borough,Neighbourhood
,,,sum
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
print (group.columns.size)

3


In [8]:
type(group)

pandas.core.frame.DataFrame

In [9]:
group.columns = column_name.iloc[0]
print(group.head())
print(group.shape)

0 Postcode      Borough                       Neighbourhood
0      M1B  Scarborough                        RougeMalvern
1      M1C  Scarborough  Highland CreekRouge HillPort Union
2      M1E  Scarborough       GuildwoodMorningsideWest Hill
3      M1G  Scarborough                              Woburn
4      M1H  Scarborough                           Cedarbrae
(103, 3)


In [10]:
for i in group['Neighbourhood']:
    ','.join(i)

In [11]:
group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# PART 2

### Sorry, I have tried tons of time but I still cannot connect to the API. So I try to use the CSV file instead.

## 1.Upload the csv

In [12]:
coo = pd.read_csv('http://cocl.us/Geospatial_data',index_col=0)

In [13]:
coo.shape

(103, 2)

In [14]:
coo.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [21]:
type(coo)

pandas.core.frame.DataFrame

In [18]:
coo.index

Index(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N',
       ...
       'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W'],
      dtype='object', name='Postal Code', length=103)

In [46]:
group_c = coo.sort_index(axis = 0,ascending = True,by = 'Postal Code')
group_c.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [52]:
for i in range(0,103):
    if group_c.index[i] == group['Postcode'][i]:
        group['Latitude'][i] = group_c['Latitude'][i]
        group['Longitude'][i] = group_c['Longitude'][i]

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [53]:
group.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,RougeMalvern,43.806686,-79.194353
1,M1C,Scarborough,Highland CreekRouge HillPort Union,43.784535,-79.160497
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [54]:
group.tail()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview VillageMartin Grove GardensRichview ...,43.688905,-79.554724
101,M9V,Etobicoke,Albion GardensBeaumond HeightsHumbergateJamest...,43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054
